In [ ]:
(version)

In [ ]:
#lang iracket/lang #:require bhdl/splicing #:reader bhdl/splicing

In [ ]:
(list (list 1 2 3) ..)

In [ ]:
(putenv "BHDL_LIBRARY_PATH" "~/bhdl/bhdl-footprints")
(getenv "BHDL_LIBRARY_PATH")

In [ ]:
(require racket/list
         racket/format
         (prefix-in pict: pict))

In [ ]:
(require bhdl)

In [ ]:
;; FIXME use global?
(define global
  (make-circuit
   #:external-pins (GND 3V3 5V VCC USB5V)))

# simple RC circuit

In [ ]:
(define (RC-fixed)
         (make-circuit
              #:external-pins (In Out GND)
              #:vars ([R1 (R '1k)]
                      [C1 (C '1p)])
          #:connect (list (*- self.In R1.1)
                          (*- R1.2 C1.1 self.Out)
                          (*- C1.2 self.Out))
          #:layout (inset (vc-append R1 C1) 20)))

In [ ]:
(show-layout (RC-fixed))

In [ ]:
(circuit-plot (RC-fixed))

# A delayed push trigger

In [ ]:
;; some component libraries
(define/IC (MOSFET)
           #:FP ((lcsc->fp "C727149") 1 2 3))

In [ ]:
(define/IC (SW_Push)
           #:FP ((kicad->fp "Button_Switch_THT.pretty/"
                                 "SW_PUSH_6mm.kicad_mod")
                 1 1 2 2)
           #:LEFT 1
           #:RIGHT 2)

In [ ]:
(define (delay-push-fixed [btn #f])
  (make-circuit
      #:external-pins (VCC GND Ctrl)
      #:vars ((RC1 (RC-fixed))
              [D1 (Diode '1N4148)]
              [R4 (R '1k)]
              [SW1 (if btn btn (SW_Push))]
              [Q1 (MOSFET '2N7002)]
              [R2 (R '10k)]
              [D2 (LED)])
      #:connect (list (*- RC1.Out D1.anode Q1.1)
                      (*- RC1.GND Q1.2)
                      (*- Q1.3 self.Ctrl))
   #:connect (list (*- self.VCC SW1 D1.cathode RC1.In self.GND)
                   (*- self.VCC D2 self.Ctrl))
   ;; TODO implement a default layout
;;    #:layout (pict:rectangle 200 200)
   #:layout (inset SW1 50)
   ))

In [ ]:
;; FIXME this does not seem to be deterministic
;; TODO implement copy-on-assign semantic, like pict?
(parameterize ([current-directory "./out/onebutton/delay-push-fixed"]
               [padding-general 2])
    (circuit-export (delay-push-fixed)
                    #:auto-place #t
                    #:formats '(kicad pdf png)))

# connect ESP-01 module

In [ ]:
(define/IC (ESP01)
           #:FP ((lcsc->fp "C503581") 
                 GND GPIO2 GPIO0 URXD
                 UTXD CH_PD RST VCC))

In [ ]:
(define/IC (OneBattery)
           ;; FIXME using pinheader for battery holder?
           #:FP ((kicad->fp "kicad-footprints" 
                           "Connector_PinHeader_2.54mm.pretty"
                           "PinHeader_1x02_P2.54mm_Horizontal.kicad_mod")
                 pos neg))

In [ ]:
(atom->fp-pict (OneBattery))

In [ ]:
(define one-button
  (make-circuit 
   #:vars ([btn (SW_Push)]
           [delay_sw1 (delay-push-fixed btn)]
           [R5 (R '10k)]
           [R6 (R '10k)]
           [BT1 (OneBattery '3V)]
           [U1 (ESP01)])
   #:connect (list 
              ;; FIXME this is not really parallel connection, but split connection
;;               (*= [BT1.pos -  U1.VCC]
;;                        [BT1.pos R5 U1.RST]
;;                        [BT1.pos R6 U1.CH_PD]
;;                        [BT1.pos -  delay_sw1.VCC])
              (*- BT1.pos (*< (*- R5 U1.RST)
                              (*- R6 U1.CH_PD)
                              U1.VCC
                              delay_sw1.VCC))
                   (*- BT1.neg U1.GND delay_sw1.GND)
                   (*- U1.RST delay_sw1.Ctrl))
   ;; TODO layout
;;    #:layout (pict:rectangle 300 300)
   #:layout (inset (hc-append 200 btn U1)
                   0 20 0 20)))

In [ ]:
(show-layout one-button)

In [ ]:
;; FIXME this does not seem to be deterministic
;; TODO implement copy-on-assign semantic, like pict?
(parameterize ([current-directory "./out/onebutton/one"]
               [padding-general 1])
    (circuit-export one-button
                    #:auto-place #t
                    #:formats '(kicad pdf png)))

# More delay pushbuttons

In [ ]:
(define three-button
  (make-circuit 
   #:vars ([buttons (for/list ([row (in-range 3)])
                              (SW_Push))]
           [button-row (for/list ([btn buttons])
                                 (delay-push-fixed btn))]
;;            [button-row (for/list ([row (in-range 3)])
;;                                 (delay-push-fixed))]
           ;; TODO use thr-hole footprint for resistors
           [R5 (R '10k)]
           [R6 (R '10k)]
           [BT1 (OneBattery '3V)]
           [U1 (ESP01)])
   #:connect (list 
              (*- BT1.pos (*< (*- R5 U1.RST)
                              (*- R6 U1.CH_PD)
                              U1.VCC))
              ;; FIXME
              (for/list ([row (in-range 3)])
                        (*- BT1.pos (pin-ref (list-ref button-row row) 'VCC)))
              ;; neg to GNDs
               (for/list ([btn button-row])
                         (*- BT1.neg U1.GND btn.GND))
              ;; connect GPIOs
              (for/list ([btn button-row])
                        (*- U1.RST btn.Ctrl))
               )
   #:layout (inset (hc-append 300 (vc-append buttons ..) U1)
                   0)))

In [ ]:
(pict:frame (show-layout three-button))

In [ ]:
;; FIXME this does not seem to be deterministic
;; TODO implement copy-on-assign semantic, like pict?
(parameterize ([current-directory "./out/onebutton/three"]
               [padding-general 1])
    (circuit-export three-button
                    #:auto-place #t
                    #:formats '(kicad pdf png)))

# Calculating RC values

In [ ]:
(define (get-C tau R)
  (/ tau R))

In [ ]:
(define (RC-flexible tau)
  (make-circuit
   #:external-pins (In Out GND)
   #:vars ([R1 (R 1000)]
           [C1 (C (get-C tau 1000))])
   #:connect (list (*- self.In R1.1)
                   (*- R1.2 C1.1 self.Out)
                   (*- C1.2 self.Out))
   #:layout (inset (vc-append R1 C1) 20)))

In [ ]:
(RC-flexible 1)

In [ ]:
;; FIXME this does not seem to be deterministic
;; TODO implement copy-on-assign semantic, like pict?
(parameterize ([current-directory "./out/onebutton/flex"]
               [padding-general 2])
    (circuit-export (RC-flexible 1)
                    #:auto-place #f
                    #:formats '(kicad pdf png bom)))

In [ ]:
;; read the CSV file
(call-with-input-file "out/onebutton/flex/BOM.csv"
                      (lambda (in)
                        (displayln (port->string in))))